In [1]:
import stackview
import numpy as np
import skimage.filters as filters
import skimage.measure as measure
import skimage.feature as feature
import skimage.segmentation as segmentation
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

In [2]:
img_dims = 256

In [3]:
img = np.zeros([img_dims, img_dims, img_dims])

In [4]:
pts = np.random.randint(0, img_dims, [1000, 3])

In [5]:
img[pts[:,0], pts[:,1], pts[:,2]] = 128

In [6]:
blurred = filters.gaussian(img, sigma=5)

In [7]:
noisy = blurred + np.random.normal(0, np.max(blurred)/100.0, img.shape)

In [8]:
stackview.slice(noisy, continuous_update=True)

In [9]:
noisy.shape

(256, 256, 256)

# Threshold and segment

In [10]:
thresh = np.percentile(noisy, 90)

In [11]:
noisy_blurred = filters.gaussian(noisy, sigma=2)

In [12]:
mask = noisy_blurred > thresh

In [13]:
edt = ndi.distance_transform_edt(mask)

In [14]:
stackview.slice(edt, continuous_update=True)

In [15]:
smoothed_edt = filters.gaussian(edt, sigma=0.1)

In [16]:
stackview.slice(smoothed_edt, continuous_update=True)

In [17]:
maxes = feature.peak_local_max(smoothed_edt, min_distance=3)

In [18]:
peak_image = np.zeros(img.shape)
peak_image[maxes[:,0], maxes[:,1], maxes[:,2]] = 1
peak_image = measure.label(peak_image)

In [19]:
watershedded = segmentation.watershed(-smoothed_edt,peak_image, mask=mask)

In [20]:
stackview.slice(watershedded)